In [300]:
import json

# Load the JSON data from data.json
with open('data/healthy_diet_foods.json') as f:
    data = json.load(f)



In [301]:
food_names = list(map(lambda f: f['name'], data['foods']))
# food_names = list(map(lambda f: f['name'], data))
food_names

['Peanut Butter, smooth',
 'Cottage cheese(Milsani, tawrog chudy | Klinek)',
 'Bananas',
 'Fish oil, cod liver',
 'Carrots, raw',
 'Chocolate, dark',
 'Milk Swieze, 2.0%',
 'Kiwifruit, raw, green',
 'Iodized salt',
 'Spinak(Szpinak Baby)',
 'Almonds',
 'Apple',
 'Yogurt',
 'Strawberries, raw',
 'Whole wheat bread',
 'Chicken, raw, meat only, boneless, skinless, breast, broiler or fryers',
 'Olive oil',
 'Raisins',
 'Pasta, dry, whole-wheat',
 "Walnuts, Trader Joe's",
 'Oatmeal',
 'Potatoes',
 'Salmon',
 'Kefir',
 'Chickpea',
 'Honey']

In [302]:
nutrients = {}
nutrients_types = set()
for food in data["foods"]:
    print(food)
    food_name = food['name']
    nutrients[food_name] = {}
    for nutrient in food['nutrients']:
        nutrients[food_name][nutrient] = food['nutrients'][nutrient]['value']
        nutrients_types.add(nutrient)

nutrients

{'name': 'Peanut Butter, smooth', 'nutrients': {'calories': {'value': 588, 'measurement': 'kkal', 'sources': ['https://www.nutritionvalue.org/Peanut_Butter%2C_smooth_nutritional_value.html?size=100+g']}, 'carbohydrates': {'value': 24, 'measurement': 'g', 'sources': ['https://www.nutritionvalue.org/Peanut_Butter%2C_smooth_nutritional_value.html?size=100+g']}, 'protein': {'value': 22, 'measurement': 'g', 'sources': ['https://www.nutritionvalue.org/Peanut_Butter%2C_smooth_nutritional_value.html?size=100+g']}, 'fat': {'value': 50, 'measurement': 'g', 'sources': ['https://www.nutritionvalue.org/Peanut_Butter%2C_smooth_nutritional_value.html?size=100+g']}, 'saturated_fat': {'value': 9.5, 'measurement': 'g', 'sources': ['https://www.nutritionvalue.org/Peanut_Butter%2C_smooth_nutritional_value.html?size=100+g']}, 'fiber': {'value': 5.7, 'measurement': 'g', 'sources': ['https://www.nutritionvalue.org/Peanut_Butter%2C_smooth_nutritional_value.html?size=100+g']}, 'vitamin_d': {'value': 0, 'measur

{'Peanut Butter, smooth': {'calories': 588,
  'carbohydrates': 24,
  'protein': 22,
  'fat': 50,
  'saturated_fat': 9.5,
  'fiber': 5.7,
  'vitamin_d': 0,
  'vitamin_a': 0,
  'iron': 2.16,
  'calcium': 54,
  'trans_fat': 0,
  'vitamin_c': 0,
  'sodium': 16,
  'zinc': 2.67,
  'vitamin_k': 0.3,
  'vitamin_b12': 0,
  'vitamin_e': 5.94,
  'potassium': 592,
  'thiamin': 0.107,
  'omega-3': 0.01,
  'omega-6': 17,
  'vitamin_b6': 0.551,
  'folate': 35,
  'riboflavin': 0.105,
  'niacin': 12.066,
  'magnesium': 179},
 'Cottage cheese(Milsani, tawrog chudy | Klinek)': {'calories': 90,
  'protein': 18,
  'carbohydrates': 3.6,
  'fat': 0.4,
  'saturated_fat': 0.3,
  'fiber': 0,
  'vitamin_d': 0.1,
  'vitamin_a': 68,
  'iron': 0,
  'calcium': 70,
  'trans_fat': 0,
  'vitamin_c': 1.1,
  'sodium': 40,
  'zinc': 0.61,
  'vitamin_k': 0.2,
  'vitamin_b12': 0.4,
  'vitamin_e': 0.08,
  'potassium': 104,
  'thiamin': 0.03,
  'omega-3': 0.02,
  'omega-6': 0.1,
  'vitamin_b6': 0.046,
  'folate': 12,
  'ribof

In [303]:
# nutrients['Cooked buckwheat groats'].get('calories', 0)

In [304]:
import pulp as pl
import copy
def find_infeasible_constraints(prob, constraints):
    for name, constraint in constraints.items():
        test_prob = copy.deepcopy(prob)
        test_prob.constraints.pop(name)
        test_prob.solve()
        if pl.LpStatus[test_prob.status] != 'Infeasible':
            print(f"Constraint {name} is likely causing infeasibility")
        # else:
        #     print(f"Constraint {name} is not causing infeasibility")

prob = pl.LpProblem("Meal_Planning_Problem", pl.LpMinimize)
food_vars = pl.LpVariable.dicts("Amount", food_names, lowBound=0)
prob += pl.lpSum([food_vars[f] for f in food_names]), "Total Weight of Food"

calories = 2800
prob += pl.lpSum([(nutrients[f].get('calories', 0) * food_vars[f]) for f in food_names] 
                    ) >= calories, "Min Calories"

def add_nutrient_constraints(prob, nutrients, food_vars, nutrient_name, min_value, max_value):
    if nutrient_name not in nutrients_types:
        raise ValueError(f"Nutrient {nutrient_name} not found in the data")
    # Adding constraints for minimum nutrient values, if specified
    if min_value is not None:
        prob += pl.lpSum([(nutrients[f].get(nutrient_name, 0) * food_vars[f]) for f in food_names] 
                         ) >= min_value, f"Min {nutrient_name}"
    if max_value is not None:
        prob += pl.lpSum([(nutrients[f].get(nutrient_name, 0) * food_vars[f]) for f in food_names] 
                         ) <= max_value, f"Max {nutrient_name}"

add_nutrient_constraints(prob, nutrients, food_vars, 'carbohydrates', calories * 0.45 / 4,  calories * 0.65 / 4)
add_nutrient_constraints(prob, nutrients, food_vars, 'protein', calories * 0.1 / 4,  calories * 0.35 / 4)
add_nutrient_constraints(prob, nutrients, food_vars, 'fat', calories * 0.0 / 9,  calories * 0.3 / 9)

add_nutrient_constraints(prob, nutrients, food_vars, 'saturated_fat', 0,  calories * 0.1 / 9)
add_nutrient_constraints(prob, nutrients, food_vars, 'trans_fat', 0,  calories * 0.01 / 9)
# add_nutrient_constraints(prob, nutrients, food_vars, 'vitamin_d', 15, 100) # mcg
add_nutrient_constraints(prob, nutrients, food_vars, 'iron', 8, 45) # mg
add_nutrient_constraints(prob, nutrients, food_vars, 'calcium', 1000, 2500) # mg
add_nutrient_constraints(prob, nutrients, food_vars, 'vitamin_c', 90, 2000) # mg
add_nutrient_constraints(prob, nutrients, food_vars, 'vitamin_a', 900, 3000) # mcg
add_nutrient_constraints(prob, nutrients, food_vars, 'fiber', 38, None) # g
add_nutrient_constraints(prob, nutrients, food_vars, 'sodium', 500, 2000) # mg
add_nutrient_constraints(prob, nutrients, food_vars, 'zinc', 11, 40) # mg
add_nutrient_constraints(prob, nutrients, food_vars, 'vitamin_k', 120, None) # mcg
add_nutrient_constraints(prob, nutrients, food_vars, 'vitamin_b12', 2.4, None) # mcg
add_nutrient_constraints(prob, nutrients, food_vars, 'vitamin_e', 15, 1000) # mg
add_nutrient_constraints(prob, nutrients, food_vars, 'potassium', 3400, 4700) # mg
add_nutrient_constraints(prob, nutrients, food_vars, 'thiamin', 1.2, None) # mg
add_nutrient_constraints(prob, nutrients, food_vars, 'omega-3', 1.6, None) # g
add_nutrient_constraints(prob, nutrients, food_vars, 'omega-6', 17, None) # g
add_nutrient_constraints(prob, nutrients, food_vars, 'vitamin_b6', 1.3, 100) # mg
# add_nutrient_constraints(prob, nutrients, food_vars, 'selenium', 55, 400) # mg
add_nutrient_constraints(prob, nutrients, food_vars, 'folate', 400, 1000) # mcg
add_nutrient_constraints(prob, nutrients, food_vars, 'riboflavin', 1.3, None) # 400 # mg, not actually established
add_nutrient_constraints(prob, nutrients, food_vars, 'niacin', 16, 35) # mg
add_nutrient_constraints(prob, nutrients, food_vars, 'magnesium', 420, None) # mg

# Add constraint for Cottage cheese(Milsani, tawrog chudy | Klinek
prob += food_vars['Cottage cheese(Milsani, tawrog chudy | Klinek)'] <= 4, "Max Cottage cheese(Milsani, tawrog chudy | Klinek"
prob += food_vars['Fish oil, cod liver'] <= 0, "Max cod liver"
prob += food_vars['Yogurt'] >= 0, "Min Yogurt"
prob += food_vars['Bananas'] >= 0, "Min Banas"
prob += food_vars['Oatmeal'] <= 0.75, "Max Oatmeal"
# prob += food_vars['Whole wheat bread'] >= 0, "Min whole wheat bread"
prob += food_vars['Whole wheat bread'] <= 1, "Max whole wheat bread"
prob += food_vars['Pasta, dry, whole-wheat'] <= 0.5, "Max Pasta, dry, whole wheat"
prob += food_vars['Potatoes'] >= 1, "Min Potatoes"


prob += food_vars['Salmon'] >= 0.5, "Min Salmon"
prob += food_vars['Apple'] <= 3, "Max Apple"
prob += food_vars['Raisins'] <= 0.4, "Max Raisins"
prob += food_vars['Chickpea'] <= 1, "Max Chickpea"
# prob += food_vars['Cottage cheese(Milsani, tawrog chudy | Klinek)'] <= 2.5, "Max Cottage cheese(Milsani)"
# prob += food_vars['Egg'] <= 1, "Max Egg"
# prob += food_vars['Steamed Broccoli'] <= 1, "Max Steamed Broccoli"
# prob += food_vars['Side Salad'] <= 1, "Max Side Salad"
# prob += food_vars['Grilled Chicken Salad'] <= 2, "Max Grilled Chicken Salad"
# prob += food_vars['Whole wheat bread'] <= 2, "Max Whole wheat bread"
# # prob += food_vars['Apple'] <= 3, "Max Apple"
# prob += food_vars['Cottage cheese(Milsani, tawrog chudy | Klinek)'] <= 2.5, "Max Cottage cheese(Milsani, tawrog chudy | Klinek)"
# prob += food_vars['Pasta, dry, whole-wheat'] <= 2, "Max Pasta, dry, whole wheat"
prob += food_vars['Almonds'] + food_vars["Walnuts, Trader Joe's"] <= 0.7, "Max Almonds and Walnuts"
prob += food_vars['Kefir'] >= 4, "Min Kefir"
prob += food_vars['Chicken, raw, meat only, boneless, skinless, breast, broiler or fryers'] <= 0, 'No meat'
prob += food_vars['Peanut Butter, smooth'] <= 0.3, 'Max Peanut Butter, smooth'
prob += food_vars['Strawberries, raw'] >= 1, 'Min Strawberries'
prob += food_vars['Honey'] <= 0.25, 'Max Honey'
prob.solve()
print("Status:", pl.LpStatus[prob.status])


print("====Products====")

for v in prob.variables():
    if v.varValue > 0:
        formatted_name = v.name.replace("Amount_", "").replace("_", " ")
        print(f"{formatted_name} = {v.varValue * 100} grams")
# Print nutrients for the optimal solution

print("====Nutrients====")
for nutrient in nutrients_types:
    nutrient_value = pl.lpSum([(nutrients[f].get(nutrient, 0) * food_vars[f].varValue) for f in food_names])
    print(f"{nutrient}: {nutrient_value}")


# Collect all constraints
print("====Constraints====")
constraints = {name: prob.constraints[name] for name in prob.constraints.keys()}
# 
# # Find infeasible constraints
# if  pl.LpStatus[prob.status] != 'Optimal':
#     find_infeasible_constraints(prob, constraints)
# https://ecosupplements.pl/product/norweski-olej-z-watroby-dorsza-1000mg-omega-3-bez-smaku-250-ml/

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/mikitashautsou/Library/Python/3.9/lib/python/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/fy/vjgx58rj0212qq321gltr5yh0000gn/T/735f054065b94f789569460ae392cb9f-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/fy/vjgx58rj0212qq321gltr5yh0000gn/T/735f054065b94f789569460ae392cb9f-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 64 COLUMNS
At line 983 RHS
At line 1043 BOUNDS
At line 1044 ENDATA
Problem MODEL has 59 rows, 26 columns and 892 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 34 (-25) rows, 24 (-2) columns and 708 (-184) elements
Perturbing problem by 0.001% of 34.286216 - largest nonzero change 0.00099435978 ( 0.086974113%) - largest zero change 0
0  Obj 6.5003893 Primal inf 333.96518 (20)
20  Obj 16.503192
Optimal - objective value 16.502176
After Postsolv

In [305]:
calories * 0.3 / 9

93.33333333333333





B1 (Thiamine): 1.1-1.2 mg/day
B2 (Riboflavin): 1.1-1.3 mg/day
B3 (Niacin): 14-16 mg/day
B5 (Pantothenic Acid): 5 mg/day
B6 (Pyridoxine): 1.3-1.7 mg/day
B7 (Biotin): 30 µg/day
B9 (Folate): 400 µg/day
B12 (Cobalamin): 2.4 µg/day
Minerals:

Calcium: 1,000-1,300 mg/day
Iron: 8-18 mg/day
Magnesium: 310-420 mg/day
Phosphorus: 700 mg/day
Potassium: 2,500-3,400 mg/day
Sodium: Less than 2,300 mg/day
Zinc: 8-11 mg/day
Iodine: 150 µg/day
Choline
selenium

Copper: 900 µg/day
Manganese: 1.8-2.3 mg/day
Fluoride: 3-4 mg/day
Antioxidants: Various amounts depending on the food source (e.g., flavonoids, polyphenols)

Phytochemicals: No specific RDI, but beneficial for health (e.g., carotenoids, flavonoids)

Probiotics: No specific RDI, but beneficial for gut health (e.g., from yogurt, kefir)

In [306]:
# Missing foods for source of:
# protein(there is a better alternative?)
# zinc 
# vitamin k(spinach could be replaced with Kale)
# b12?(though milk is already included)
# vitamin e add Sunflower Seeds?
# thiamin source?
# omega-6: can be optimized, walnuts has not the highest level of the nutrient 

## Constraints to add
Vitamin E: Nuts, seeds, spinach, broccoli.
Folate (Vitamin B9): Leafy green vegetables, legumes, and fortified cereals.
Niacin (Vitamin B3): Poultry, fish, beef, and whole grains.
Vitamin B6: Fish, poultry, potatoes, and bananas.
Vitamin B12: Meat, fish, dairy products, and fortified cereals.

Potassium: Bananas, oranges, potatoes, and tomatoes.
Manganese: Whole grains, nuts, legumes, and leafy green vegetables.


Check constraints



# Fats (especially essential fatty acids) V time spent: 2 hours
    omega 3 V
    omega 6 V
# Vitamin C V
# Vitamin D V
# Vitamin A V
# B vitamins (
        B12, V
        B6, V (but Pistachio nuts can be added as the highest content product)
        folate(add chickpeas?) V,
        thiamin, V
        riboflavin V,
        niacin М
        )
# Iron V
# Calcium V
# Zinc V
# Magnesium
# Potassium V
# Iodine
# Selenium
# Vitamin E V
# Vitamin K V
# Phosphorus
# Manganese
# Copper
# Chromium
# Molybdenumz
https://claude.ai/chat/bb3a5672-5950-4f41-bef8-c76eb1e75d97